In [1]:
import time

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn import datasets, preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
iris = datasets.load_iris()

In [4]:
X = preprocessing.normalize(iris.data[:, :2],axis=1)  # we only take the first two features.
y = iris.target

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [6]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, max_iter=1, warm_start=True,
                    hidden_layer_sizes=(5), random_state=1)

In [7]:
def get_all_weights(clf):
    all_weights = [] 
    for layer in clf.coefs_:
        layers_weights = layer.reshape(1,-1)
        all_weights = np.append(all_weights, layers_weights)
    return all_weights

In [8]:
import colorsys

def val_to_rgb(val):
    i = val*0.67
    c = colorsys.hsv_to_rgb(i,1,1)
    return [int(color*255) for color in c]

In [9]:
from led_api import led_api

def set_leds_to_weights(weights):
    led_data_to_send = {"leds":[]}
    for i,w in np.ndenumerate(weights):
        norm_weight = max(-1, min(w, 1))+1
        (r,g,b) = val_to_rgb(norm_weight)
        to_add = { "red": r, "green": g, "blue": b, "led_num": i[0] }
        led_data_to_send['leds'].append(to_add)
    led_api.send_led_json(led_data_to_send)
    
def clear_leds():
    led_data_to_send = {"leds":[]}
    for i in range(30):
        to_add = { "red": 0, "green": 0, "blue": 0, "led_num": i }
        led_data_to_send['leds'].append(to_add)
    led_api.send_led_json(led_data_to_send)

In [10]:
# clear led strip
clear_leds()

In [11]:
for i in range(10):
    time.sleep(0.1)
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    set_leds_to_weights(get_all_weights(clf))

0.3
0.32
0.32
0.3
0.32
0.32
0.3
0.32
0.32
0.34
